Привет, Глеб!)
<br> Меня зовут Дуолан 👋 Буду проверять твой проект. Давай вместе доведем его до идеала 😉
<br> Дальнейшее общение будет происходить на «ты», если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий («исправил», «не понятно как исправить ошибку», ...)
<br> Пожалуйста, не удаляй мои комментарии, они будут необходимы для повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>❌ Замечание:</b> Если требуются исправления
</div>

Работа не может быть принята с красными комментариями.

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:

<div class="alert alert-block alert-info">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-info">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [ ]:
!pip install category_encoders

## Подготовка данных

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
import numpy as np
import decimal

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PowerTransformer, StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.james_stein import JamesSteinEncoder 
from category_encoders.count import CountEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline

pd.options.display.float_format = '{:.2f}'.format
%matplotlib

In [ ]:
df = pd.read_csv('/datasets/autos.csv')

In [ ]:
df.shape

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Импорт выглядит отлично 👍
</div>

DateCrawled — дата скачивания анкеты из базы
VehicleType — тип автомобильного кузова
RegistrationYear — год регистрации автомобиля
Gearbox — тип коробки передач
Power — мощность (л. с.)
Model — модель автомобиля
Kilometer — пробег (км)
RegistrationMonth — месяц регистрации автомобиля
FuelType — тип топлива
Brand — марка автомобиля
Repaired — была машина в ремонте или нет
DateCreated — дата создания анкеты
NumberOfPictures — количество фотографий автомобиля
PostalCode — почтовый индекс владельца анкеты (пользователя)
LastSeen — дата последней активности пользователя

In [ ]:
df.sample(5)

In [ ]:
df.dtypes

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()

In [ ]:
len(df['PostalCode'].unique())

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

От явных дубликатов избавились 👍
</div>

Представляется, что дата скачивания анкеты с базы данных (DateCrawled), дата последней активности пользователя (LastSeen), почтовый индекс (PostalCode), дата публикации объявления (DateCreated), месяц регистрации (RegistrationMonth) сами по себе не представляют интереса.

На основе данных почтового индекса (PostalCode) можно было бы выделить населенный пункт жительства продавца, но так как из какой страны эти данные неизвестно, то сделать это не представляется возможным. Также использовать их в качестве категориальных признаков, по-моему, тоже нельзя, так как их слишком много.

In [ ]:
new_columns = list(df.columns)
for i in ['DateCrawled', 'LastSeen', 'PostalCode', 'RegistrationMonth', 'DateCreated']:
    new_columns.remove(i)

In [ ]:
df = df[new_columns]

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Удалить неинформативные признаки - хорошее решение 👍
</div>

In [ ]:
df.dtypes

In [ ]:
df.columns = ['price', 'vehicle_type', 'registration_year', 'gearbox', 'power',
              'model', 'kilometer', 'fuel_type', 'brand', 'repaired', 'number_pictures']

In [ ]:
df.describe()

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> 

Работать с такими названиями гораздо удобнее)
</div>

number_pictures везде ноль, поэтому этот столбец также необходимо удалить. 

In [ ]:
df.drop('number_pictures', axis=1, inplace=True)

In [ ]:
cat_feat = df.dtypes[df.dtypes == object].index
num_feat = df.dtypes[df.dtypes != object].index

In [ ]:
for i in num_feat:
    print(i, df[i].median())

### Обработка пропусков

In [ ]:
df.isnull().sum()

In [ ]:
df[df.model.isnull()].isnull().sum()

Так как модель автомобиля является значимой характеристикой, по-моему мнению, стоит удалить те строки, где она не указана. 

In [ ]:
df.dropna(subset=['model'], inplace=True)

In [ ]:
df.isnull().sum()

Остальные пропуски стоит пометить, как 'unknow'.

In [ ]:
df.fillna('unknow', inplace=True)

In [ ]:
df.isnull().sum()

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Пропуски успешно заполнены 👍 Модели теперь не будут капризничать)
</div>

Похоже, что количественные данные содержат выбросы поэтому постараемся их найти при помощи графиков

In [ ]:
cat_feat

In [ ]:
num_feat

In [ ]:
for feat_name in num_feat:
    print(f'{feat_name}:')
    fig = px.box(df, y=feat_name)
    fig.show()

Имеются следующие величины имеющие выбросы: цена, год регистрации, мощность.

Небольшой пробег автомобиля при его продаже нельзя считать выбросом, так как такие факты бывают. 

Представляется необходимым сразу разобратся с очевидными выбрасами, такими как 0 скорость и мощность, а также неадекватный год регистрации.

In [ ]:
print(sum((df.power == 0)))
print(sum((df.price == 0)))

Значения с 0 явно неадекватные, поэтому их необходимо удалить. 

In [ ]:
df.drop(df[(df.power == 0) | (df.price == 0)].index, axis=0, inplace=True)

In [ ]:
df.shape

In [ ]:
df['registration_year'].value_counts()

In [ ]:
fig = px.histogram(df, x="registration_year")
fig.show()

Исходя из распределения данных представляется необходимым преобразовать числовую характеристику годов в категориальную по следующему принципу: 
- 1960-1980; 
- 1980-1990; 
- 1990-2000; 
- 2000-2010;
- 2010-2015;
- 2015-2020; 
- иные не попавшие не в один из диапозонов. 

Таким образом, мы преобразуем числовую характеристику годов в категориальную и выделим выбросы в отдельную категорию не удаляя данные с указанными выбросами. 

In [ ]:
def classific_year(x):
    if x >= 1960 and x < 1980:
        return '60-80'
    elif x >= 1980 and x < 1990:
        return '80-90'
    elif x >= 1990 and x < 2000:
        return '90-00'
    elif x >= 2000 and x < 2010:
        return '00-10'
    elif x >= 2010 and x < 2015:
        return '10-15'
    elif x >= 2015 and x < 2020:
        return '15-20'
    else:
        return 'other'

In [ ]:
df['registration_year'] = df['registration_year'].apply(classific_year)

In [ ]:
df['registration_year'] = df['registration_year'].astype('category')

In [ ]:
df['registration_year'].value_counts()

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Изящно 👍
</div>

In [ ]:
df.dtypes

In [ ]:
fig = px.histogram(df['registration_year'])
fig.show()

<div class="alert alert-block alert-info">
Вопрос. У меня вроде бы уже не тип данных дата, но график строется как-то странно.. с чем это может быть связано?
</div>

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

Думаю, нужно, чтобы тип данных должен быть object)
</div>

In [ ]:
fig = px.histogram(df, x="price")
fig.show()

Распределение цены выглядит странно, цены как бы делятся на свои собственные распределения со своими локальными пиками.
Также имеется большое количество цен с 0. 

In [ ]:
df.shape

In [ ]:
sum((df.price < 400))

Представляется, что цена автомобиля стоимостью менее 400$ является выбросом, который надо удалить. 

In [ ]:
df = df[df.price >= 400]

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

В данных остались машины только с адекватной ценой 👍
</div>

In [ ]:
sum(df.price > 15000)

In [ ]:
df.columns

In [ ]:
df.loc[df.price > 15000, 'model'].value_counts()

In [ ]:
df['class'] = df['price'].apply(lambda x: 'more' if x >= 15000 else 'less')

In [ ]:
fig = px.histogram(df, x="power", color='class')
fig.show()

In [ ]:
more = df.loc[df.price >= 15000, 'power']
less = df.loc[df.price < 15000, 'power']
for data, name in zip([more, less], ['more', 'less']):
    print(f'Медиана {name}: {data.median()}')
    print(f'Среднее {name}: {data.mean()}')

Как видно из представленных графиков, а также среднего и медианны, мощность автомобилей с ценой более 15 тыс. долларов больше чем для других автомобилей, таким образом очень дорогие автомобили это не выброс. 

In [ ]:
df.dtypes

В ходе предобработки удалены признаки: LastSeen, PostalCode, DateCreated, RegistrationMonth, number_pictures. Удалены выбросы в следующих признаках: price, power. Признак registration_year преобразован в категориальный.

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> Данные готовы для обучения 👍
</div>

## Обучение моделей

В ходе работы обучим линейную регрессию, LigtGBM, Catboost.

In [ ]:
X = df.drop('price', axis=1)
y = df['price']
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.8)
cat_feat = list(X_train.dtypes[X_train.dtypes != int].index)
num_feat = list(X_train.dtypes[X_train.dtypes == int].index)

In [ ]:
cat_feat

In [ ]:
num_feat

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> 

Данные поделены верно, пропорции выбраны адекватные 👍
</div>

### LinearRegression

In [ ]:
numeric_transformer = Pipeline(steps=[
     ("scaler", StandardScaler())])


categorical_transformer = Pipeline(steps=[
    ('cat', OneHotEncoder(handle_unknown="value"))
])

preprocessor = ColumnTransformer(transformers=[
     ("num_transform", numeric_transformer, num_feat),
    ("cat_transform", categorical_transformer, cat_feat)    
])


pipeline_linear = Pipeline([('preprocessor', preprocessor), 
                    ('clf', Lasso())])
pipeline_linear

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> 

Для линейных моделей прямое кодирование отлично подойдет 👍
</div>

In [ ]:
list_alpha = [0.5, 1, 2]
param = [ {"clf__alpha": list_alpha}
        ]  
                                       
grid_search_linear = GridSearchCV(pipeline_linear, param, cv=5, n_jobs=-1, 
                                 refit='rmse', scoring='neg_mean_squared_error')
grid_search_linear.fit(X_train, y_train)
print("Лучшие параметры:")
print(grid_search_linear.best_params_)
print("Лучшая метрика на валидационных данных:")
print(grid_search_linear.best_score_)

Я ошибку выше никак побороть не могу... (ValueError: Input contains NaN, infinity or a value too large for dtype('float64')). Это проблема касается линейных моделей (аналогичный результат на Ridge), на градиентном бустинге все норм. 

<div class="alert alert-block alert-info">
<b>Комментарий ревьюера:</b>

Думаю дело в масштабировании. Попробуй выполнить масштабирование вручную через StandardScaler. И масштабировать только численные признаки 
</div>

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> 

Модель обучена корректно 👍
</div>

### Catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
model = CatBoostRegressor(iterations=2000, 
                          cat_features=cat_feat,
                          verbose=False
                         )
param = {'l2_leaf_reg':[0.5, 1], 
        'depth':[6, 8, 9], 
         'learning_rate':[0.03, 0.003]
       }

grid_search = model.grid_search(param,
                                X=X_train,
                                y=y_train,
                                train_size=0.8,
                                refit=True,
                                cv=3,
                                calc_cv_statistics=True,
                                verbose=False,
                                plot=True)

<div class="alert alert-block alert-info">
<b>ВОПРОС:</b>

Модель явно переобучена (на валидационных и тренировочных данных метрика качества сильно разная), но переобучение все равно приводит к улучшению метрики. Переобученность модели в таком случае стоит игнорировать? Или в таких случаях стоит повышать регулиризацию?

<div class="alert alert-block alert-info">
<b>V2 Комментарий ревьюера:</b>

На тренировочных данных она всегда будет показывать результат лучше, чем на валидационной) Если метрика на тестовой адекватна, то модель можно считать адекватной
</div>

In [ ]:
grid_search['params']

Обучим заново модель с лучшими параметрами, чтобы найти время за которое она обучится. 

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> 

Модель обучена корректно 👍
</div>

### LGBMRegressor

In [ ]:
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [ ]:
X_train_lg = X_train.copy()

In [ ]:
X_train_lg[cat_feat] = X_train_lg[cat_feat].apply(LabelEncoder().fit_transform)

In [ ]:
params = {
    'num_leaves': [31, 50],
    'learning_rate': [0.03, 0.003],
    'max_depth': [-1, 5],
    'n_estimators': [500, 1000],
}

grid_lg = GridSearchCV(LGBMRegressor(), params, scoring='neg_root_mean_squared_error', cv=3)
grid_lg.fit(X_train_lg, y_train)

In [ ]:
print("Лучшие параметры:")
print(grid_lg.best_params_)
print("Лучшая метрика на валидационных данных:")
print(grid_lg.best_score_)

Чтобы узнать время обучения обучим заново модель LGBMRegressor, включим туда время предобработки категориальных фичей, для чистоты эксперимента. 

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b> 

Модель обучена корректно 👍
</div>

## Анализ моделей

In [ ]:
table = pd.DataFrame(grid_search_linear.cv_results_)
linear_list = list(table.loc[table.params == grid_search_linear.best_params_, ['mean_fit_time', 'mean_score_time']].values[0])
linear_list.append(abs(grid_search_linear.best_score_))
linear_list

In [ ]:
tabel_lgbmr = pd.DataFrame(grid_lg.cv_results_)
lgbmr_list = list(tabel_lgbmr.loc[tabel_lgbmr.params == grid_lg.best_params_, ['mean_fit_time', 'mean_score_time']].values[0])
lgbmr_list.append(abs(grid_lg.best_score_))
lgbmr_list

<div class="alert alert-block alert-info">

grid_search катбуста данных о среднем времени обучения и предсказания не предаставляет, поэтому обучим модель заново на лучших параметрах. И используем при предсказании X_test **только** для того, что бы узмерить время предсказания. 

In [ ]:
%%time
model_cat = CatBoostRegressor(iterations=2000, 
                          cat_features=cat_feat,
                          depth=9,
                          learning_rate=0.03,
                          l2_leaf_reg=0.5,
                          verbose=False
                         )
model_cat.fit(X_train, y_train, verbose=False)

In [ ]:
%%time
model_cat.predict(X_test)

In [ ]:
cat_list = [264, 5.44, 1853]

In [ ]:
result = pd.DataFrame([linear_list, lgbmr_list, cat_list], 
                      columns=['mean_fit_time', 'mean_score_time', 'best_score'], 
                      index=['linear', 'LGBMRegressor', 'Catboost'])

In [ ]:
result

<div class="alert alert-block alert-danger">
<b>❌ Замечание:</b>
    
Анализ моделей не должен включать тестовую выборку. Тестовая выборка используется в самом конце - в финальном тестировании лучшей модели. 

Если валидациционной выборки нет, то анализ моделей должен выглядеть так:

1. RMSE вычисляется с помощью кросс-валидации на тренировочной выборке
2. Время обучения = время model.fit(X_train, y_train)
3. Время предсказания = model.predict(X_train)
</div>

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b>

Приятно было бы видеть результаты в информативной таблице
</div>

<div class="alert alert-block alert-warning">
<b>⚠️ Совет:</b>

Из того же GridSearchCV мы можем получить время обучения и время предсказания)

В атрибуте `cv_results_` есть среднее время обучения и среднее время предсказания для всех комбинаций.
</div>

<div class="alert alert-block alert-success">
<b>V2 ✔️ Успешно исправлено</b>
</div>

## Вывод

В ходе анализа установлено следующее: 
- время обучения catboost 4 мин. 20 сек., время предсказания для тестовых данных: 4.82 сек., метрика RMSE 1619.87; 
- время обучения LGBM 20 сек., время предсказания для тестовых данных: 29.9 сек., метрика RMSE 1648.04.
        
Выбор модели зависит от наличных ресурсов и потребностей заказчика. Предполагаю, что целесообразно выбрать catboost, потому что он быстрее предсказывает и поэтому работу такой модели, возможно, будет проще интегрировать в сайт. Также более продолжительное время обучения, чем у LGBM не должно быть проблемой, если размер предполагаемых тренировочных данных заказчика не больше в 1000 раз и не будет возможности обучить модель на GPU.

<div class="alert alert-block alert-success">
<b>✔️ Успех:</b>

Вывод соответствует исследованию 👍
</div>

<div class="alert alert-block alert-info">
Исходя из данных представленных ниже LGBM является наилучшим выбором, возможно catboost при обучении на GPU справится лучше. 

In [ ]:
result

# <font color='orange'>Общее впечатление</font>
* Этот проект выполнен очень хорошо
* Видно, что приложено много усилий
* Молодец, что структурируешь ноутбук, приятно проверять такие работы
* У тебя чистый и лаконичный код
* Мне было интересно читать твои промежуточные выводы
* Твой уровень подачи материала находится на высоком уровне
* Исправь, пожалуйста, мои замечания. Затем отправляй на повторную проверку
* Жду новую версию проекта 👋

# <font color='orange'>2. Общее впечатление</font>
* Спасибо за быстрое внесение правок
* Теперь проект выглядит лучше )
* Критических замечаний нет
* Молодец, отличная работа!
* Надеюсь, ревью было полезным
* Удачи в дальнейшем обучении 👋

# <font color='orange'>Рекомендации 🔥</font>
* Доступное объяснение про работу градиентного бустинга https://www.youtube.com/watch?v=ZNJ3lKyI-EY&t=995s
* После просмотра видео можешь почитать статью про ансамбли и бустинги https://habr.com/ru/company/ods/blog/645887/
* Подбор гиперпараметров для CatBoost https://habr.com/ru/company/otus/blog/527554/
* Подбор гиперпараметров для LightGBM https://programmersought.com/article/40776531516/
* Разумные способы кодирования категориальных данных для машинного обучения https://machinelearningmastery.ru/smarter-ways-to-encode-categorical-data-for-machine-learning-part-1-of-3-6dca2f71b159/
* Статья про "непонятную" метрику ROC-AUC https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/
* Хочешь подтянуть математику для DS?) https://academy.stepik.org/math-for-data-science
* Результы обучения моделей можно визуализировать 😎 https://www.datatechnotes.com/2019/08/elasticnet-regression-example-in-python.html
* В нашем деле нужно быть всегда в курсе всех новинок, новостей и тд, вот тут публикуют новости в области DS: https://www.infoq.com/data-analysis/news/
* Всем аналитикам данных рекомендую книгу Даниела Канемана "Думай медленно, решай быстро"